In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import datetime
import importlib.machinery
import os
import sys
import time
import yaml

from pprint import pprint

In [3]:
sys.path.append("/home/caleml/main-pe/")

In [6]:
import numpy as np

from data.datasets.h36m import Human36M
from data.loader import BatchLoader
from data.utils import transform, camera
from data.utils.data_utils import TEST_MODE, TRAIN_MODE, VALID_MODE

from model import config, measures
from model.utils import pose_format, log

## dataset

In [7]:
# local loading
local_h36m_path = '/home/caleml/datasets/h36m'
local_h36m = Human36M(local_h36m_path, dataconf=config.human36m_dataconf, poselayout=pose_format.pa17j3d, topology='frames') 

In [8]:
h36m_val = BatchLoader(local_h36m, 
                       ['frame'],
                       ['pose_w', 'pose_uvd', 'afmat', 'camera'], 
                       VALID_MODE,
                       batch_size=local_h36m.get_length(VALID_MODE), 
                       shuffle=True)

log.printcn(log.OKBLUE, 'Preloading Human3.6M validation samples...')

[x_val], [pw_val, puvd_val, afmat_val, scam_val] = h36m_val[0]

Preloading Human3.6M validation samples...


In [9]:
dataset = {
    'x_val': x_val,
    'pw_val': pw_val,
    'puvd_val': puvd_val,
    'afmat_val': afmat_val,
    'scam_val': scam_val
}

## eval func

In [10]:
def eval_human36m_sc_error(model,
                           x,
                           pose_w,
                           afmat,
                           rootz,
                           scam,
                           resol_z=2000.,
                           batch_size=8,
                           map_to_pa17j=None,
                           logdir=None,
                           verbose=True):

    assert len(x) == len(pose_w) == len(afmat) == len(scam)

    num_blocks = len(model.outputs)

    y_true_w = pose_w.copy()
    if map_to_pa17j is not None:
        y_true_w = y_true_w[:, map_to_pa17j, :]
    y_pred_w = np.zeros((num_blocks,) + y_true_w.shape)
    if rootz.ndim == 1:
        rootz = np.expand_dims(rootz, axis=-1)

    pred = model.predict(x, batch_size=batch_size, verbose=1)

    # Move the root joints from GT poses to origin
    y_true_w -= y_true_w[:, 0:1, :]

    if verbose:
        log.printc(log.WARNING, 'Avg. mm. error:')

    lower_err = np.inf
    scores = []

    for b in range(num_blocks):

        if num_blocks > 1:
            y_pred = pred[b]
        else:
            y_pred = pred

        # ??
        y_pred = y_pred[:, :, 0:3]

        # project normalized coordiates to the image plane
        y_pred[:, :, 0:2] = transform.transform_pose_sequence(afmat.copy(), y_pred[:, :, 0:2], inverse=True)

        # Recover the absolute Z
        y_pred[:, :, 2] = (resol_z * (y_pred[:, :, 2] - 0.5)) + rootz
        y_pred_uvd = y_pred[:, :, 0:3]

        # camera inverse projection
        for j in range(len(y_pred_uvd)):
            cam = camera.camera_deserialize(scam[j])
            y_pred_w[b, j, :, :] = cam.inverse_project(y_pred_uvd[j])

        # Move the root joint from predicted poses to the origin
        y_pred_w[b, :, :, :] -= y_pred_w[b, :, 0:1, :]

        err_w = measures.mean_distance_error(y_true_w[:, 0:, :], y_pred_w[b, :, 0:, :])
        scores.append(err_w)
        if verbose:
            log.printc(log.WARNING, ' %.1f' % err_w)

        # Keep the best prediction
        if err_w < lower_err:
            lower_err = err_w

    if verbose:
        log.printcn('', '')

    if logdir is not None:
        np.save('%s/y_pred_w.npy' % logdir, y_pred_w)
        np.save('%s/y_true_w.npy' % logdir, y_true_w)

    log.printcn(log.WARNING, 'Final averaged error (mm): %.3f' % lower_err)

    return scores

# model folder

In [ ]:
# single h5 eval
weights_path = '/home/caleml/pe_experiments/test_from_gpuserver2/weights_036.h5'
eval_model = MultiBranchModel(dim=3, n_joints=17, nb_pose_blocks=1)
eval_model.load_weights(weights_path, pose_only=True)
scores = eval_human36m_sc_error(eval_model.model, 
                                x_val, 
                                pw_val, 
                                afmat_val,
                                puvd_val[:,0,2], 
                                scam_val,  
                                batch_size=24)

pprint(scores)

In [11]:
# whole folder eval
def eval_folder(exp_folder, dataset, pose_only=False):
    config_path = os.path.join(exp_folder, 'config.yaml')
    
    # config
    with open(config_path, 'r') as f_conf:
        config = yaml.load(f_conf)
        
    # model
    # mb_module = importlib.machinery.SourceFileLoader('multi_branch_model', os.path.join(exp_folder, 'model_src/model/networks/multi_branch_model.py')).load_module()
    # print("Model module loaded from %s" % mb_module.__file__)
    # eval_model = mb_module.MultiBranchModel(dim=3, n_joints=17, nb_pose_blocks=int(config['pose_blocks']))
    from model.networks.multi_branch_model import MultiBranchModel
    eval_model = MultiBranchModel(dim=3, n_joints=17, nb_pose_blocks=int(config['pose_blocks']))
        
    # find weights paths
    weights = list()
    for filename in os.listdir(exp_folder):
        if filename.endswith(".h5") and filename.startswith("weights_"):
            weights.append(os.path.join(exp_folder, filename))
            
    weights.sort()
    print("Found %s weights paths" % len(weights))
    
    # actual eval
    all_scores = list()
    for i, weights_path in enumerate(weights):
        time1 = time.time()
        print("Eval of weights_path %s" % weights_path)
        eval_model.load_weights(weights_path, pose_only=pose_only)
        
        scores = eval_human36m_sc_error(eval_model.model, 
                                        dataset['x_val'], 
                                        dataset['pw_val'], 
                                        dataset['afmat_val'],
                                        dataset['puvd_val'][:,0,2], 
                                        dataset['scam_val'],  
                                        batch_size=24)
        
        time2 = time.time()
        print("Scores for epoch %s: %s (in %s s)" % (i + 1, str(scores), time2 - time1))
        all_scores.append(max(scores))
        
    return all_scores
        

In [ ]:
exp_baseline_path = '/home/caleml/pe_experiments/exp_baseline_1b_bs32_h36m_201903221052/'
all_scores_baseline = eval_folder(exp_baseline_path, dataset, pose_only=True)

In [12]:
exp_hybrid_path = '/home/caleml/pe_experiments/exp_20190322_1942_hybrid_h36m__1b_bs16/'
# clean_pythonpath(exp_folder=exp_hybrid_path)
all_scores_hybrid = eval_folder(exp_hybrid_path, dataset, pose_only=False)

gud_layers
gud_blocks
Found 60 weights paths
Eval of weights_path /home/caleml/pe_experiments/exp_20190322_1942_hybrid_h36m__1b_bs16/weights_001.h5


/home/caleml/.local/lib/python3.4/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


gud_pose
pose shape (?, 17, 3), vis shape (?, 17, 1), concat shape (?, 17, 4)
Last H shape Tensor("fReMap1/batch_normalization_13/cond/Merge:0", shape=(?, 32, 32, 576), dtype=float32)
gud_dec
Build E_a 7.58323335647583, build E_p 5.7701873779296875, decoder D 0.6277205944061279
Input shape (?, 256, 256, 3)
Shape z_a (?, 16, 16, 1024), shape z_p (?, 16, 16, 1024)
Outputs shape [(None, 256, 256, 3), (None, 17, 4)]
rec y_pred shape (?, 256, 256, 3)
pose y_pred shape (?, 17, 4)
Shape (?, ?, ?)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
model (Model)                   (None, 16, 16, 1024) 8589184     input_12[0][0]                   
__________

ValueError: axes don't match array

In [ ]:
def clean_pythonpath(exp_folder=None):
    print('before path %s' % str(sys.path))
    clean_path = [p for p in sys.path if 'pe_experiments' not in p]
    
    if '/home/caleml/main-pe/' in clean_path:
        clean_path.remove('/home/caleml/main-pe/')
        
    if '' in clean_path:
        clean_path.remove('')
    
    if exp_folder:
        clean_path.insert(0, os.path.join(exp_folder, 'model_src/'))
    sys.path = clean_path
    print('after path %s' % str(sys.path))
    
    
def restore_pythonpath():
    clean_pythonpath()
    
    if '' not in sys.path:
        sys.path.insert(0, '')
    
    if '/home/caleml/main-pe/' not in sys.path:
        sys.path.append('/home/caleml/main-pe/')
    

In [ ]:
restore_pythonpath()
print(sys.path)

## plot

In [ ]:
import matplotlib.pyplot as plt

x = list(range(60))

plt.plot(x, all_scores_baseline, 'r')
# plt.plot(x, all_scores_hybrid, 'b')
plt.title('Baseline val')
plt.xlabel('epoch')
plt.ylabel('MPJPE')
plt.show()

plt.savefig('/home/caleml/pe_experiments/exp_baseline_1b_bs32_h36m_201903221052/eval.png')

In [ ]:
all_scores_baseline = [91.75194501441577, 82.46205898898975, 79.4226132617545, 88.8355976647399, 84.02042960060331, 86.88580619918093, 86.67989807960359, 79.48412115594127, 78.59867146077222, 84.09675808749151, 76.1932848291697, 77.14141621003114, 78.2065916383256, 81.43181095386677, 78.58923745192591, 77.008543664543, 77.46245838987596, 76.31208539465088, 78.66618899639796, 75.53613154051648, 83.53373459959678, 76.01499659875537, 74.4797050967049, 79.84342586353483, 77.25320791938644, 76.69853350783292, 76.45083687520106, 77.09345226614492, 75.78290824659078, 77.9041891354044, 78.09753993393933, 76.60042799938726, 77.5805151579404, 78.8091096371011, 74.80468494705836, 74.2166559806093, 77.63991530864764, 77.89325147099139, 78.38807773666701, 80.47055450518361, 73.81642579401903 , 75.0416006683364 , 74.48196930787806 , 75.15854590724823 , 73.66802279838019 , 75.19603076463348 , 74.01143213366338 , 75.08591589680957 , 75.55668131643323 , 77.11394826271881 , 74.44167918712098 , 75.63973677387979 , 76.3378921014415 , 74.73289168541717 , 73.95828566807859 , 75.55537095370421 , 75.78301989366678 , 74.91063809506264 , 74.59803512567959 , 74.75948044066446]